# Päringuvastuste kvaliteedi hindamine

Meil ei ole otseselt võimalik hinnata päringulaiendaja töökvaliteeti, sest me ei tea millised on tüüpilised Riigi Teatajale tehtud päringud.
Samas saame me hinnata milliste päringusõnade korral on laiendatud otsisõnade arv väga suur ja seega otsing ise aeglane. 

In [2]:
import sqlite3
from pandas import DataFrame

con = sqlite3.connect('../results/query_extender_setup/rt_titles.sqlite')
cur = con.cursor()

## I. Korrektsetele sisendsõnadele vastavate laiendatud otsisõnade jaotus

Esmalt vaatame, millistel korrektsetel sõnavormidel on kõige rohkem vormilisi laiendusi. See võib sõnavormiti erineda, sest ühele sõnavormile võib vastata mitu algvormi ja sellekaudu mitme erineva algvormi tekstides esinevad vormid. 

Näited kõige suurema vastete arvuga sisendvormidest.

In [51]:
threshold = 4
result = cur.execute(
    """
    SELECT 
        lemma_kõik_vormid.vorm as sisend_sõna,
        lemma_korpuse_vormid.vorm as väljund_päring
    FROM lemma_kõik_vormid
    JOIN lemma_korpuse_vormid
    ON lemma_kõik_vormid.lemma = lemma_korpuse_vormid.lemma
    WHERE lemma_kõik_vormid.vorm in 
    (
        SELECT sisend_sõna
        FROM
        (
            SELECT
            	sisend_sõna,
            	count(*) as väljundite_arv
            FROM
            (
            	SELECT 
            		lemma_kõik_vormid.vorm as sisend_sõna,
            		lemma_korpuse_vormid.vorm as väljund_päring
            	FROM lemma_kõik_vormid
            	JOIN lemma_korpuse_vormid
            	ON lemma_kõik_vormid.lemma = lemma_korpuse_vormid.lemma
            )
            GROUP BY sisend_sõna
        )
        WHERE väljundite_arv >= {THRESHOLD})
        """.format(THRESHOLD=threshold)
)
problematic_inputs = (DataFrame(result.fetchall(), columns=['sisendvorm', 'väljundvorm'])
                      .groupby('sisendvorm', as_index=False).aggregate(vasted=('väljundvorm', list))
                      .assign(vastete_arv=lambda df: df['vasted'].map(len))
                      .sort_values(['vastete_arv', 'sisendvorm'], ascending=[False, True])
                     )

display(problematic_inputs)

,sisendvorm,vasted,vastete_arv
0,kogu,"[kogu, kogude, kokku, kogu]",4
1,kogud,"[kogu, kogude, kokku, kogu]",4
2,kogudes,"[kogu, kogude, kokku, kogu]",4
3,koguks,"[kogu, kogude, kokku, kogu]",4
4,kogus,"[kogu, kogude, kokku, kogu]",4
5,kogusid,"[kogu, kogude, kokku, kogu]",4
6,koguta,"[kogu, kogude, kokku, kogu]",4
7,kokku,"[kogu, kogude, kokku, kokku]",4
8,koole,"[kooli, koolis, koolis, koos]",4


## II. Kirjavigadega sisendsõnadele vastavate laiendatud otsisõnade jaotus

Kõik kirjavigadega ostsisõnavormid ei anna vasteid: osad sisendid on liiga vigased ja teised vormid on küll parandatavad, kuid neile vastavaid algvorme ei esine tekstides. Seetõttu saame vaadata vaid vastetega kirjaveavorme.

In [65]:
result = cur.execute(
    """
    SELECT 
        väljundite_arv,
        count(*) as sisenvormide_arv
    FROM
    (
        
        SELECT
            sisendvorm,
            count(*) as väljundite_arv
        FROM
        (
            SELECT
                vigane_vorm as sisendvorm,
                vorm as väljundvorm
            FROM
            (
                SELECT 
                    vigane_vorm,
                    lemma
                FROM kirjavead
                JOIN lemma_kõik_vormid
                ON kirjavead.vorm = lemma_kõik_vormid.vorm
            ) as tbl1
            JOIN lemma_korpuse_vormid
            ON tbl1.lemma = lemma_korpuse_vormid.lemma 
        )
        GROUP BY sisendvorm
    )
    GROUP BY väljundite_arv
""")

summary = (DataFrame(result.fetchall(), columns=['väljundite_arv', 'sisendvormide_arv'])
           .assign(osakaal_protsentides=lambda df: df['sisendvormide_arv']/sum(df['sisendvormide_arv'])*100).round(2))
display(summary)

,väljundite_arv,sisendvormide_arv,osakaal_protsentides
0,1,2729,86.42
1,2,296,9.37
2,3,104,3.29
3,4,26,0.82
4,5,2,0.06
5,8,1,0.03


Näited kõige suurema vastete arvuga sisendvormidest.

In [71]:
threshold = 5
result = cur.execute(
    """
    SELECT 
        sisendvorm,
        väljundvorm
    FROM
    (
        SELECT
            vigane_vorm as sisendvorm,
            vorm as väljundvorm
        FROM
        (
            SELECT 
                vigane_vorm,
                lemma
            FROM kirjavead
            JOIN lemma_kõik_vormid
            ON kirjavead.vorm = lemma_kõik_vormid.vorm
        ) as tbl1
        JOIN lemma_korpuse_vormid
        ON tbl1.lemma = lemma_korpuse_vormid.lemma     
    
    )
    WHERE sisendvorm in 
    (
        SELECT
            sisendvorm
        FROM
        (
            SELECT
                sisendvorm,
                count(*) as vastete_arv
            FROM
            (
                SELECT
                    vigane_vorm as sisendvorm,
                    vorm as väljundvorm
                FROM
                (
                    SELECT 
                        vigane_vorm,
                        lemma
                    FROM kirjavead
                    JOIN lemma_kõik_vormid
                    ON kirjavead.vorm = lemma_kõik_vormid.vorm
                ) as tbl1
                JOIN lemma_korpuse_vormid
                ON tbl1.lemma = lemma_korpuse_vormid.lemma 
            )
            GROUP BY sisendvorm
        )
        WHERE vastete_arv >= {THRESHOLD}
    )
    """.format(THRESHOLD=threshold)
)

problematic_inputs = (DataFrame(result.fetchall(), columns=['sisendvorm', 'väljundvorm'])
                      .groupby('sisendvorm', as_index=False).aggregate(vasted=('väljundvorm', list))
                      .assign(vastete_arv=lambda df: df['vasted'].map(len))
                      .sort_values(['vastete_arv', 'sisendvorm'], ascending=[False, True])
                     )

display(problematic_inputs)                      


,sisendvorm,vasted,vastete_arv
1,koku,"[kogu, kogude, kokku, kogu, kogu, kogude, kokk...",8
0,esti,"[Eesti, eesti, Eesti, eesti, eesti]",5
2,ürgi,"[Türgi, türgi, Türgi, türgi, türgi]",5


In [14]:
result = cur.execute(
    """
    SELECT
    	väljundite_arv,
    	count(*) as sisendvormide_arv
    FROM	
    ( 
        SELECT
        	sisend_sõna,
        	count(*) as väljundite_arv
        FROM
        (
        	SELECT 
        		lemma_kõik_vormid.vorm as sisend_sõna,
        		lemma_korpuse_vormid.vorm as väljund_päring
        	FROM lemma_kõik_vormid
        	JOIN lemma_korpuse_vormid
        	ON lemma_kõik_vormid.lemma = lemma_korpuse_vormid.lemma
        )
        GROUP BY sisend_sõna
    )
    GROUP BY väljundite_arv
    """
)
summary = (DataFrame(result.fetchall(), columns=['väljundite_arv', 'sisendvormide_arv'])
           .assign(osakaal_protsentides=lambda df: df['sisendvormide_arv']/sum(df['sisendvormide_arv'])*100).round(2))
display(summary)

,väljundite_arv,sisendvormide_arv,osakaal_protsentides
0,1,4663,93.30
1,2,295,5.90
2,3,31,0.62
3,4,9,0.18
